<a href="https://colab.research.google.com/github/koll-ai/infinitecanvas_client/blob/main/ic_client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook from https://github.com/koll-ai/infinitecanvas_client**

[Discord](https://discord.gg/yvZKgSUzaW)

In [4]:
#@title # Install dependencies

from IPython.utils import capture

with capture.capture_output() as cap:
  !npm install -g localtunnel
  !pip install diffusers
  !pip install transformers
  !pip install accelerate
  !git clone https://github.com/koll-ai/infinitecanvas_client.git

import subprocess
import time
from infinitecanvas_client.imagen import ImageGenerator
from flask import Flask
from flask import request
import base64
from io import BytesIO

In [2]:
#@title # Choose the model you want to use with koll.ai
Version = "runwayml/stable-diffusion-v1-5" #@param [ "runwayml/stable-diffusion-v1-5", "runwayml/stable-diffusion-inpainting", "stabilityai/stable-diffusion-2-1", "stabilityai/stable-diffusion-2", "stabilityai/stable-diffusion-2-inpainting"]

#@markdown Or
Path = "" #@param {type:"string"}

model_name = Path if Path != "" else Version

imagen = ImageGenerator()
with capture.capture_output() as cap:
  imagen.load_model(model_name, 'cuda')

In [ ]:
#@title # Run the service with localtunnel

with open('output.txt', 'w') as f:
    proc = subprocess.Popen(['lt', '--port', '5000'], stdout=f)

time.sleep(3)

app = Flask(__name__)

@app.route('/', methods=['POST'])
def index():
    json = request.get_json()
    action = json['action']
    params = json['params']

    image = None

    if action == 'new_image':
      image = imagen.new_image(**params)
    
    elif action == 'outpainting':
      image = imagen.outpainting(**params)[0]

    elif action == 'img_to_img':
      image = imagen.image_to_image(**params)

    buffered = BytesIO()
    image.save(buffered, format="PNG")
    
    return base64.b64encode(buffered.getvalue())

if __name__ == '__main__':
  with open('output.txt', 'r') as f:
      contents = f.read()
      API_URL = contents.split(' ')[-1]
      protocol = API_URL.split(':')[0]
      url = API_URL.split('/')[-1]

      print('\n'*2)
      print(f'Everything is ready! Click on the this link to be redirected to koll.ai. Don\'t close this tab!')
      print(f'https://koll.ai?protocol={protocol}&url={url}')

with capture.capture_output() as cap:
  app.run()